# Learning to Use a Hyper-parameter optimizer

In [2]:
import sherpa
import rescomp as rc
import numpy as np

TOL = 0.2
EXPERIMENT = ("lorenz", 15, 0.01, None, 0)
DEFAULTS = {
    "res_sz":100,
    "activ_f": lambda x: 1/(1+np.exp(-1*x)),
    "sparse_res":True,
    "uniform_weights":True,
    "signal_dim":3,
    "max_weight":2,
    "min_weight":0,
    "batchsize":2000
}

In [3]:
def rcompacc(system, duration, dt, window, overlap, **kwargs):
    tr, Utr, ts, Uts = train_test_orbit(system, duration=duration, dt=dt)
    rcomp = rc.ResComp(**kwargs)
    rcomp.train(tr, Utr)
    r0 = rcomp.r0
    pre = rcomp.predict(ts, r0=r0)
    idx = accduration(Uts, pre, tol=TOL)
    acc = ts[idx-1] - ts[0]
    return acc

In [4]:
def train_test_orbit(system, duration=10, dt=0.01):
    t, U = rc.orbit(system, duration=duration, dt=dt)
        # Train and test data
    N = len(t)
    mid = int(N//2)
    tr, Utr = t[:mid], U[:mid, :]
    ts, Uts = t[mid:], U[mid:, :]
    return tr, Utr, ts, Uts

In [5]:
def L2(x, axis=0):
    return np.sum(x**2, axis=axis)**0.5
def Linf(x, axis=0):
    return np.max(np.abs(x), axis=axis)

def relerr(true, pre, order=2, axis=0):
    if order == 2:
        norm = L2
    if order == "inf":
        norm = Linf
    return norm(true - pre, axis=axis) / norm(true, axis=axis)

def accduration(true, pre, tol=0.2, order="inf", axis=0):
    n = pre.shape[axis]
    for i in range(n):
        if axis == 0:
            t = true[i, :]
            p = pre[i, :]
        if axis == 1:
            t = true[:, i]
            p = pre[:, i]
        if relerr(t, p, order=order, axis=0) > tol:
            return i
    return n

def system_fit_error(t, pre, system, order=2):
    df = rc.SYSTEM[system]["df"]
    trueprime = df(t, pre)
    dt = np.mean(np.diff(t))
    fdprime = np.gradient(pre, dt, axis=1)
    err = relerr(trueprime, fdprime)
    if order == 2:
        return np.mean(err)
    if order == inf:
        return np.max(err)
    return err

## Hyper Parameter Opt

In [6]:
parameters = [sherpa.Continuous(name='gamma', range=[0.1, 25], ),
              sherpa.Continuous(name='sigma', range=[0.01, 5.0]),
              sherpa.Continuous(name='spect_rad', range=[0.1, 25]),
              sherpa.Continuous(name='ridge_alpha', range=[1e-8, 2], scale='log'),
              sherpa.Continuous(name='mean_degree', range=[0.1, 5])]

algorithm = sherpa.algorithms.RandomSearch(max_num_trials=150)

study = sherpa.Study(parameters=parameters,
                 algorithm=algorithm,
                 lower_is_better=False)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running. Access via
http://127.0.0.1:8880 or 
http://DJs-MacBook-Pro.local:8880 if on a cluster, or 
http://localhost:8880 if running locally.
-------------------------------------------------------


 * Serving Flask app "sherpa.app.app" (lazy loading)
 * Debug mode: on
 * Environment: production
   Use a production WSGI server instead.


In [7]:
num_iterations = 10
for trial in study:
    for i in range
    acc = rcompacc(*EXPERIMENT, **trial.parameters, **DEFAULTS)
    study.add_observation(trial=trial, objective=acc)
    study.finalize(trial)

KeyboardInterrupt: 

In [8]:
study.get_best_result()

{'Trial-ID': 2,
 'Iteration': 6,
 'gamma': 0.8840808597489546,
 'mean_degree': 4.421765603834887,
 'ridge_alpha': 2.659321927333624e-05,
 'sigma': 2.031273657486462,
 'spect_rad': 9.788853067936978,
 'Objective': 12.5}

In [10]:
study.results

,Trial-ID,Status,Iteration,gamma,mean_degree,ridge_alpha,sigma,spect_rad,Objective
0,1,INTERMEDIATE,0,9.572021,4.726406,4.290003e-06,3.407161,13.592568,0.32
1,1,INTERMEDIATE,1,9.572021,4.726406,4.290003e-06,3.407161,13.592568,0.13
2,1,INTERMEDIATE,2,9.572021,4.726406,4.290003e-06,3.407161,13.592568,0.12
3,1,INTERMEDIATE,3,9.572021,4.726406,4.290003e-06,3.407161,13.592568,0.46
4,1,INTERMEDIATE,4,9.572021,4.726406,4.290003e-06,3.407161,13.592568,0.03
...,...,...,...,...,...,...,...,...,...
1645,150,INTERMEDIATE,6,1.400819,3.080906,5.461505e-08,0.970869,16.485947,0.30
1646,150,INTERMEDIATE,7,1.400819,3.080906,5.461505e-08,0.970869,16.485947,0.06
1647,150,INTERMEDIATE,8,1.400819,3.080906,5.461505e-08,0.970869,16.485947,0.21
1648,150,INTERMEDIATE,9,1.400819,3.080906,5.461505e-08,0.970869,16.485947,0.28


In [11]:
import pandas as pd

In [13]:
df = study.results
df.aggregate(np.mean,"Tiral-ID",)

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed

In [ ]:
study.save()